<a href="https://colab.research.google.com/github/imji0319/PDSH/blob/master/nyc_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training XGBoost Model
### nyc_0.feather 데이터로 training 한 model을 초기값으로 1-20까지의 파일 모두 반복 실행


In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [0]:
pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)


In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.model_selection import train_test_split 
import xgboost as xgb
from xgboost import Booster

In [0]:
def XGBmodelsaved(x_train,x_test,y_train,y_test,params,tmpFile):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=300, xgb_model=tmpFile,verbose_eval=50,
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

In [0]:
#Cross-validation
params = {
    # Parameters that we are going to tune.
    'max_depth': 8, #Result of tuning with CV
    'eta':.03, #Result of tuning with CV
    'subsample': 1, #Result of tuning with CV
    'colsample_bytree': 0.8, #Result of tuning with CV
    # Other parameters
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

In [0]:
# 분할된 데이터 로드 및 train 

%%time
for i in range(1,21):
    path = 'nyc_'+str(i)+'.feather'
    df_train =pd.read_feather(path)
    df_train = df_train.drop(columns=['level_0', 'index'])
    y = df_train['fare_amount']
    train = df_train.drop(columns=['fare_amount'])
    
    del df_train
    
    X_train, X_test, y_train, y_test = train_test_split(train, y, random_state = 100, test_size=0.25)
    
    del train,y
    
    fname="xgb.model"
    bst = xgb.Booster()
    tmpFile = bst.load_model(fname)
    
    model = XGBmodelsaved(X_train,X_test,y_train,y_test,params,tmpFile)
    print(str(i) + '번 파일데이터가 트레이닝 되었습니다.')
    
    model.save_model('xgb.model')
    del model
    

[07:16:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	test-rmse:14.0148
Will train until test-rmse hasn't improved in 10 rounds.
[50]	test-rmse:4.75852
[100]	test-rmse:3.64173
[150]	test-rmse:3.52219
[200]	test-rmse:3.48509
[250]	test-rmse:3.46222
[299]	test-rmse:3.44776
1번 파일데이터가 트레이닝 되었습니다.
[0]	test-rmse:13.9325
Will train until test-rmse hasn't improved in 10 rounds.
[50]	test-rmse:4.59483
[100]	test-rmse:3.43593
[150]	test-rmse:3.31143
[200]	test-rmse:3.27378
[250]	test-rmse:3.25242
[299]	test-rmse:3.23604
2번 파일데이터가 트레이닝 되었습니다.
[0]	test-rmse:13.9339
Will train until test-rmse hasn't improved in 10 rounds.
[50]	test-rmse:4.58484
[100]	test-rmse:3.41192
[150]	test-rmse:3.28795
[200]	test-rmse:3.24906
[250]	test-rmse:3.2282
[299]	test-rmse:3.21288
3번 파일데이터가 트레이닝 되었습니다.
[0]	test-rmse:13.9622
Will train until test-rmse hasn't improved in 10 rounds.
[50]	test-rmse:4.6056
[100]	test-rmse:3.43294
[150]	test-rmse:3.30853
[200]	test-rmse:3.26904
[250]	test-rmse:3.24427
[299]	test-rmse:3.22242
4번 파일데이터가 트레이닝 되었습니다.
[0]	test-rmse:13.9252
Will

In [0]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [0]:
fname="xgb.model"

bst = xgb.Booster(model_file=fname)

In [0]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
df_test = pd.read_csv('cleaning_test.csv')

In [0]:
test_key = df_test['key']
x_pred = df_test.drop(columns=['key', 'pickup_datetime'])

In [0]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,hour,distance_miles,distance_to_center
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,13,1.443607,3.841863
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,13,1.507044,1.084252
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1,2011,5,11,0.384398,2.851254
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1,2012,5,21,1.218529,3.934481
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1,2012,5,21,3.347514,5.634652


In [0]:
#Predict from test set
prediction = bst.predict(xgb.DMatrix(x_pred))

In [0]:
submission = pd.DataFrame({'key' : df_test.key,
                          'fare_amount': prediction},
                         columns =['key','fare_amount'])

submission.head(10)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.876581
1,2015-01-27 13:08:24.0000003,11.115511
2,2011-10-08 11:53:44.0000002,4.732394
3,2012-12-01 21:12:12.0000002,7.972251
4,2012-12-01 21:12:12.0000003,13.869342
5,2012-12-01 21:12:12.0000005,9.793613
6,2011-10-06 12:10:20.0000001,5.125116
7,2011-10-06 12:10:20.0000003,48.913635
8,2011-10-06 12:10:20.0000002,11.696170
9,2014-02-18 15:22:20.0000002,6.690121


In [0]:
submission.to_csv('all_xgb_submission.csv', index=False)